Plan: use more data overlap

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

import numpy as np
from random import random, uniform, randrange, triangular

In [2]:
import os
files = os.listdir("dataset")

import csv
from tqdm import tqdm
class Route:
    def __init__(self, phi, v, M):
        self.phi = phi
        self.v = v
        self.M = M
    def __len__(self):
        return len(self.phi)
    
routes = []

allRows = 0

maxAngle = 0
maxSpeed = 0
maxTorque = 0

for file in files:
    path = os.path.join("dataset", file)
    with open(path, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ')
        phi = [] # Steering angle
        v = [] # Speed
        M = [] # Output torque
        for row in spamreader:
            phi.append(float(row[1]))
            v.append(float(row[2]))
            M.append(float(row[3]))

            maxAngle = max(abs(float(row[1])), maxAngle)
            maxSpeed = max(abs(float(row[2])), maxSpeed)
            maxTorque = max(abs(float(row[3])), maxTorque)
        routes.append(Route(np.array(phi, dtype=np.float32), np.array(v, dtype=np.float32), np.array(M, dtype=np.float32)))
        allRows += len(phi)


print(allRows)
print(maxAngle, maxSpeed, maxTorque)

330394
16.049999237060547 34.94847106933594 1.0


Now we have everything stored in routes. Time to generate some test data

In [3]:
#Pytorch imports
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [4]:
class SimDataset(Dataset):
    def __init__(self, length, i, o):
        self.len = length
        self.i = i
        self.o = o

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.i[idx], self.o[idx]

prev_data = 300
fwd_data = 100

input_length = 3 * prev_data + fwd_data

def get_sample(test):
    if test:
        route = routes[-1]
    else:
        route = routes[randrange(len(routes)-1)]

    idx = randrange(prev_data, len(route.v)-fwd_data)
    
    prev_angle = route.phi[idx-prev_data : idx]/maxAngle
    prev_speed = route.v[idx-prev_data : idx]/maxSpeed
    torque = route.M[idx-prev_data : idx+fwd_data]/maxTorque

    output = route.phi[idx+fwd_data-1 : idx+fwd_data]/maxAngle
    
    return prev_angle, prev_speed, torque, output

all_routes = len(routes)
train_routes = routes[0:12]
test_routes = routes[12:14]
def caddy_dataset(train):
    i_data = []
    o_data = []
    routes = train_routes if train else test_routes
    for route in tqdm(routes):
        for idx in range(prev_data, len(route.v)-fwd_data+1):
            phi = route.phi[idx-prev_data : idx]/maxAngle
            v = route.v[idx-prev_data : idx]/maxSpeed
            M = route.M[idx-prev_data : idx+fwd_data]/maxTorque

            o = route.phi[idx+fwd_data-1 : idx+fwd_data]/maxAngle
        
            data_input = np.concatenate((phi, v, M))
            i_data.append(data_input)
            o_data.append(o)
    return SimDataset(len(i_data), i_data, o_data)


train_set = caddy_dataset(True)
test_set = caddy_dataset(False)

print(len(train_set), len(test_set))

100%|██████████| 2/2 [00:00<00:00,  2.43it/s]

265961 58847


In [5]:
train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_set, batch_size=64, shuffle=False)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_length, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.linear_relu_stack(x)


In [7]:
def train_loop(dataloader, model, loss_fn, optimizer, debug):
    size = len(dataloader.dataset)
    for X, y in dataloader:
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if debug:
        loss = loss.item()*100
        print(f"Train loss: {loss}")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()*100

    test_loss /= num_batches
    print(f"Test  loss: {test_loss} \n")

model = NeuralNetwork()
for p in model.parameters():
    print(p.shape)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=20e-6)

torch.Size([64, 1000])
torch.Size([64])
torch.Size([64, 64])
torch.Size([64])
torch.Size([32, 64])
torch.Size([32])
torch.Size([1, 32])
torch.Size([1])
70337


In [8]:
epochs = 10
print(len(train_dataloader))
print(len(test_dataloader))
skip = 1
for t in range(epochs):
    if t%skip == 0:
        print(f"Epoch {t}")
    train_loop(train_dataloader, model, loss_fn, optimizer, t%skip==0)
    if t%skip == 0:
        test_loop(test_dataloader, model, loss_fn)

4155
920
Epoch 0
Train loss: 0.07940682698972523
Test  loss: 0.07215757551589179 

Epoch 1
Train loss: 0.060639751609414816
Test  loss: 0.05824343114182818 

Epoch 2
Train loss: 0.02678383025340736
Test  loss: 0.05251650865212152 

Epoch 3
Train loss: 0.03740957472473383
Test  loss: 0.053637174022321196 

Epoch 4
Train loss: 0.055832212092354894
Test  loss: 0.05347414407493568 

Epoch 5
Train loss: 0.05182572640478611
Test  loss: 0.049476446691622426 

Epoch 6
Train loss: 0.04824564966838807
Test  loss: 0.04584019391949784 

Epoch 7
Train loss: 0.04608226881828159
Test  loss: 0.0510163951503273 

Epoch 8
Train loss: 0.03812775248661637
Test  loss: 0.0459034840221885 

Epoch 9
Train loss: 0.04244059382472187
Test  loss: 0.04711439707628652 



In [45]:
random_sample = test_set[randrange(len(test_set))]
print(random_sample[0][299]*maxAngle, "initial angle")
print(random_sample[1]*maxAngle)

torch_data_input = torch.from_numpy(random_sample[0])
print(model(torch_data_input).item()*maxAngle)
print(f"Loss: {(model(torch_data_input).item()*maxAngle-random_sample[1]*maxAngle)**2}")

0.15000000306770644 initial angle
[0.45]
0.8348597092445402
Loss: [0.14811702]


In [13]:
#Make a plot
random_sample = test_set[randrange(len(out_set))]
i = random_sample[0]
v = i[0:300]*maxSpeed
phi = i[300:600]*maxAngle
M = i[600:1000]*maxTorque
v.resize(400)
phi.resize(400)
phi[-1] = random_sample[1][0]*maxAngle

time = 400
_, axs = plt.subplots(3)
axs[0].plot(np.linspace(0, time, time), v)
axs[1].plot(np.linspace(0, time, time), phi)
axs[2].plot(np.linspace(0, time, time), M)

print(random_sample[0][599]*maxAngle, "initial angle")
print(random_sample[1]*maxAngle)

torch_data_input = torch.from_numpy(random_sample[0])
print(model(torch_data_input).item()*maxAngle)

NameError: name 'out_set' is not defined

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))
print(model.eval())
test_loop(test_dataloader, model, loss_fn)